En la tarea de esta entrega trabajará cuatro aplicaciones de procesamiento del lenguaje general: clasificación de texto, generación de texto, análisis sintáctico y recuperación de información (_Information Retrieval_) . 

Las funciones que necesitará las puede obtener del código del capítulo de NLP del libro AIMA. Deberá mirar qué dependencias tienen respecto a otros archivos, incluir el código que sea necesario o importarlo de otros módulos.

[https://github.com/aimacode/aima-python/blob/master/nlp_apps.ipynb](https://github.com/aimacode/aima-python/blob/master/nlp_apps.ipynb)

# 1. Clasificación de texto según el autor

Construya dos modelos de diferentes autores: Josep Carner y Miquel dels Sants Oliver. Para eso, puede usar sus obras disponibles en el Proyecto Gutenberg. Por ejemplo, de Carner tome la traducción de los cuentos de Mark Twain. Después, clasifica frases en el estilo de cada uno que muestren cómo su modelo las identifica correctamente.

In [7]:
from aima_module.utils import open_data
from aima_module.text import words, UnigramWordModel
from aima_module.learning import NaiveBayesLearner

def recognize_author(text, nbs):
   return nbs(words(text.lower()))

hostel = open_data('CA-Text/bolla_hostel.txt').read()
wordseq_hostel = words(hostel)
p_miquel = UnigramWordModel(wordseq_hostel, 5)

adventures = open_data('CA-Text/tom_sawyer_adventures.txt').read()
wordseq_adventures = words(adventures)
p_josep = UnigramWordModel(wordseq_adventures, 5)


nbs = NaiveBayesLearner({('Miquel', 1): p_miquel, ('Josep', 1): p_josep}, simple=True)

In [8]:
# From the "L'hostal de la Bolla" [line 172]
recognize_author('de la familia, entre sacs de bessó i senalles de llegum. La botiga,', nbs)

'Miquel'

In [9]:
# From "The Adventures of Tom Sawyer" [line 4149]
recognize_author('resta de cent yardes. Nedava quartejant per amunt el corrent, però', nbs)

'Josep'

![Author's quotes prompt](./authors_phrases_prompt.png)

In [10]:
# Quote from Miquel dels Sants Oliver
recognize_author('La joventut, en el sentit màgic, potser no ha mai existit i potser no és sinó un equador convencional entre la il·lusió i el record.', nbs)

'Josep'

In [11]:
# Quote from Josep Carner
recognize_author('Una mica d\'amistat fins a morir diu que dura. Vés, amor, metzina impura; vull la mel, vull el brossat d\'una mica d\'amistat.', nbs)

'Josep'

Como comentario a cerca del resultado, se puede observar que no es del todo preciso en la clasificación. Haciendo uso de la actividad de mi compañero David Ramírez ([enlace a su tarea](https://github.com/davidzz-code/IEDIB_2024_2025/blob/main/02%20Modelos%20IA/T5%20NLP/tarea_4_David_Ramirez_Ruiz.ipynb)), voy a probar con las frases que ha utilizado él.

In [12]:
# Quote from Miquel dels Sants Oliver
recognize_author('Els carrers, encara humits, semblaven desertar de la nit', nbs)

'Miquel'

In [13]:
# Quote from Josep Carner
recognize_author('Amb un gest majestuós em fità un instant', nbs)

'Josep'

Podemos ver que en este caso, el clasificador si que los diferencia correctamente.

# 2. Generación de texto

Construya modelos n-gram con n=1, n=3, n=5 y=7 a partir de un texto elegido por usted. Puede ser una obra del Proyecto Gutenberg o una noticia de prensa, por ejemplo. Observe como medida que aumenta el número de palabras que se han tenido en cuenta la verosimilitud del texto generado es mayor.

In [21]:
from random import choice
from aima_module.text import NgramWordModel, words

def generate_text(model, length, ngram=5):
	generate = []
	seed = choice(list(model.dictionary.keys()))
	generate.extend(seed)

	for i in range(length-ngram):
		last = tuple(generate[-(ngram-1):])
		next = []

		for j in model.dictionary.keys():
			if j[:ngram-1] == last: next.append(j[ngram-1])
		
		if not next: break

		next = choice(next)
		generate.append(next)
	
	return ' '.join(generate)

# In my case I decided to use the Spanish classic 'Fuenteovejuna' by Lope de Vega
fuenteovejuna = open_data('ES-Text/fuente_ovejuna.txt').read()
wordseq = words(fuenteovejuna)

# I decided to automate the text generation process from 1 ngram to 10 ngrams and 100 words length
for i in range(1, 11):
	model = NgramWordModel(i, wordseq)
	print(f'Ngram: {i}\n{generate_text(model=model, length=100, ngram=i)}\n\n')

Ngram: 1
solamente


Ngram: 2
eso viva mil insultos hac a hijo esteban mirad los dos jacinta apelo de suerte en lo averig e unless you within 30 2019 ebook for nearly all viene con cuidado buena para vuestros hombros flacos mirad los pa os laurencia all comendador as celoso de puro amor a refund set forth in formats readable form accessible by all vuestra gente el desear sale la quit is all en materia mengo as con leve causa se hiela barrildo es ya le merece y ya que habemos de tu consejo con paciencia y son extremadas laurencia bravo y antes un corcillo


Ngram: 3
24 madrid como tantas otras comedias de nuestro teatro la presente publicada en 1619 los par ntesis cuadrados indican que se trata de un viejo de palos das laurencia si le faltare alg n corzo esc ndete en esas ramas frondoso y laurencia laurencia se or es ste laurencia tirando viene a ser posible a la edad esteban qui resle t laurencia voluntad le he cobrado pero por lo que nos quit is el valor amor hay y principal de alguno

Como se esperaba, a medida que se aumentan los _n-gram_ los textos son más coherentes, aunque se pueden ver ciertos casos como en el **ngram=8** que no es perfecto.

# 3. Análisis sintáctico

Aplica las herramientas de alguna librería Python para analizar sintácticamente la siguiente oración:

- (ES): _Todos los seres humanos nacen libres e iguales en dignidad y derechos._

- (CA): _Tots els éssers humans neixen lliures i iguals en dignitat i drets._

In [14]:
!python -m spacy download ca_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 38.5 MB/s eta 0:00:0031m41.7 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')


In [15]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 4.1 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [16]:
import spacy

es_nlp = spacy.load('es_core_news_sm')
ca_nlp = spacy.load('ca_core_news_sm')

es_sentence = 'Todos los seres humanos nacen libres e iguales en dignidad y derechos'
ca_sentence = 'Tots els éssers humans neixen lliures i iguals en dignitat i drets'

es_doc = es_nlp(es_sentence)
ca_doc = ca_nlp(ca_sentence)

In [17]:
# Catalan sentence syntax analysis
print('- CATALAN Syntax -')
print('Noun Phrases:', [chunk.text for chunk in ca_doc.noun_chunks])
print('Verbs:', [token.lemma_ for token in ca_doc if token.pos_ == 'VERB'])
print('Subjects:', [token.text for token in ca_doc if token.dep_ == 'nsubj'])
print('Adjectives:', [token.text for token in ca_doc if token.pos_ == 'ADJ'])
print('Prepositions:', [token.text for token in ca_doc if token.pos_ == 'ADP'])
print('Conjunctions:', [token.text for token in ca_doc if token.pos_ == 'CCONJ'])
print('+ Each word Information:')
for token in ca_doc:
	print(f'Word: {token.text} | Lemma: {token.lemma_} | Part of Speech: {token.pos_} | Syntactic Dependency: {token.dep_} | Head: {token.head.text} \n')


- CATALAN Syntax -
Noun Phrases: ['Tots els éssers humans', 'dignitat i drets']
Verbs: ['neixar']
Subjects: ['éssers']
Adjectives: ['humans', 'lliures', 'iguals']
Prepositions: ['en']
Conjunctions: ['i', 'i']
+ Each word Information:
Word: Tots | Lemma: tot | Part of Speech: DET | Syntactic Dependency: det | Head: els 

Word: els | Lemma: el | Part of Speech: DET | Syntactic Dependency: det | Head: éssers 

Word: éssers | Lemma: ésser | Part of Speech: NOUN | Syntactic Dependency: nsubj | Head: neixen 

Word: humans | Lemma: humà | Part of Speech: ADJ | Syntactic Dependency: amod | Head: éssers 

Word: neixen | Lemma: neixar | Part of Speech: VERB | Syntactic Dependency: ROOT | Head: neixen 

Word: lliures | Lemma: lliure | Part of Speech: ADJ | Syntactic Dependency: obj | Head: neixen 

Word: i | Lemma: i | Part of Speech: CCONJ | Syntactic Dependency: cc | Head: iguals 

Word: iguals | Lemma: igual | Part of Speech: ADJ | Syntactic Dependency: conj | Head: lliures 

Word: en | Lemma:

In [18]:
# Spanish sencence syntax analysis
print('- SPANISH Syntax -')
print('Noun Phrases:', [chunk.text for chunk in es_doc.noun_chunks])
print('Verbs:', [token.lemma_ for token in es_doc if token.pos_ == 'VERB'])
print('Subjects:', [token.text for token in es_doc if token.dep_ == 'nsubj'])
print('Adjectives:', [token.text for token in es_doc if token.pos_ == 'ADJ'])
print('Prepositions:', [token.text for token in es_doc if token.pos_ == 'ADP'])
print('Conjunctions:', [token.text for token in es_doc if token.pos_ == 'CCONJ'])
print('+ Each word Information:')
for token in es_doc:
	print(f'Word: {token.text} | Lemma: {token.lemma_} | Part of Speech: {token.pos_} | Syntactic Dependency: {token.dep_} | Head: {token.head.text} \n')


- SPANISH Syntax -
Noun Phrases: ['Todos los seres humanos', 'dignidad', 'derechos']
Verbs: ['nacer']
Subjects: ['seres']
Adjectives: ['humanos', 'libres', 'iguales']
Prepositions: ['en']
Conjunctions: ['e', 'y']
+ Each word Information:
Word: Todos | Lemma: todo | Part of Speech: DET | Syntactic Dependency: det | Head: los 

Word: los | Lemma: el | Part of Speech: DET | Syntactic Dependency: det | Head: seres 

Word: seres | Lemma: ser | Part of Speech: NOUN | Syntactic Dependency: nsubj | Head: nacen 

Word: humanos | Lemma: humano | Part of Speech: ADJ | Syntactic Dependency: amod | Head: seres 

Word: nacen | Lemma: nacer | Part of Speech: VERB | Syntactic Dependency: ROOT | Head: nacen 

Word: libres | Lemma: libre | Part of Speech: ADJ | Syntactic Dependency: obj | Head: nacen 

Word: e | Lemma: e | Part of Speech: CCONJ | Syntactic Dependency: cc | Head: iguales 

Word: iguales | Lemma: igual | Part of Speech: ADJ | Syntactic Dependency: conj | Head: libres 

Word: en | Lemma: e

# 4. Recuperación de información

Cargue el artículo de la wikipedia sobre Europa ([https://es.wikipedia.org/wiki/Europa](https://es.wikipedia.org/wiki/Europa)) en una lista de frases y recupera la información siguiente, buscando las frases más parecidas.

- ¿Cuándo se gestó el concepto de Europa?

- ¿Cuál es la especie humana autóctona de Europa?

- ¿Cuándo se formaron los estados actuales de Europa?

- ¿Qué clima tiene Europa?

¿Todas las preguntas obtienen una respuesta adecuada? ¿Hay preguntas que conviene formular de otra forma?

---

[Repository Link](https://github.com/CharlyMech/IEDIB_CEIA_2024-25/blob/main/Modelos%20de%20Inteligencia%20Artificial/Bloque%205/Actividades/5.1/tarea_5_Carlos_SanchezRecio.ipynb)